# 야구 뉴스 댓글 정보 추출

2018, 2019 댓글 덤프
댓글에 해당하는 뉴스 원문 덤프
원문에서 인터뷰 기사 분리
  감독 인터뷰
  선수 인터뷰
  기타
댓글이 달린 기사 원문 분석

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import bz2
import pandas as pd

from datetime import datetime
from io import BufferedWriter
from elasticsearch import Elasticsearch
from tqdm.autonotebook import tqdm
from utils.naver_reply_utils import NaverNewsReplyUtils
from utils.elasticsearch_utils import ElasticSearchUtils

pd.set_option('display.max_colwidth', -1)

In [ ]:
host = 'https://corpus.ncsoft.com:9200'
auth = 'crawler:crawler2019'

reply_utils = NaverNewsReplyUtils()

reply_utils.init_plt()

# 2018, 2019 댓글 덤프
news_index = 'crawler-naver-sports-2019'
reply_index = 'crawler-naver-sports-reply-2019'

In [ ]:
def dump_data(news_index, reply_index):
    """ """
    reply_list = reply_utils.dump_data(
        host=host,
        auth=auth,
        doc_type='reply',
        index_list=[reply_index],
    )

    reply_info = reply_utils.flatten_reply(reply_list=reply_list)

    news_df = reply_utils.get_news(
        host=host,
        auth=auth,
        index=news_index,
        reply_idx=reply_info['index'],
        reply_list=reply_info['docs'],
    )

    reply_df = pd.DataFrame(reply_info['data'])

    # 파일로 저장
    reply_utils.save_json(df=news_df, filename=news_index + '.json.bz2')
    reply_utils.save_json(df=reply_df, filename=reply_index + '.json.bz2')
    
    return news_df, reply_df

news_df, reply_df = dump_data(news_index=news_index, reply_index=reply_index)

In [ ]:
news_df = reply_utils.read_json(filename=news_index + '.json.bz2')
reply_df = reply_utils.read_json(filename=reply_index + '.json.bz2')

# 전체 인용문 추출 

In [ ]:
len(quote['quote']), len(quote['quote_news']), len(news_df)

# 2018: (150,488, 44,633, 305,831)
# 2019: (90,869, 26,593, 196,364)

In [ ]:
# 오류 분석

In [ ]:
freq = quote_utils.remove_low_freq(
    df=quote_df,
    by='who',
    min_freq=1,
)

In [ ]:
pd.set_option('display.max_rows', 200)

freq['freq'].sort_values(0, ascending=False)[
    (0 < freq['freq'][0]) & (freq['freq'][0] <= 1)
]

In [ ]:
quote_df[
    quote_df['who'] == ''
]

In [ ]:
clean = quote_utils.remove_low_freq(
    df=quote_df,
    by='who',
    min_freq=1,
)

quote_df = clean['clean_df']
clean['low_freq']

In [ ]:
clean = quote_utils.remove_low_freq(
    df=quote_df,
    by='verb',
    min_freq=1,
)

quote_df = clean['clean_df']
clean['low_freq']

In [ ]:
quote_utils.save_json(df=quote_df, filename=news_index + '.quote.json.bz2')
# quote_df.to_excel('naver-sport-quote.2019.xlsx')

In [ ]:
quote_utils.info(
    news_df=news_df,
    reply_df=reply_df,
    quote_df=quote_df,
)

In [ ]:
column = 'text'
text = '미세먼지'

quote_df[ quote_df[column].apply(lambda x: '\n'.join(x)).str.contains(text) ]

In [ ]:
news_df[news_df['doc_id'] == '468-0000480959']

In [ ]:
utils.get_quote(news_df[news_df['doc_id'] == doc_id])

In [ ]:
pd.set_option('display.max_colwidth', -1)

# quote_df[quote_df['who'] == '그']
news_df[news_df['doc_id'] == '477-0000191360']

In [ ]:
pd.set_option('display.max_colwidth', -1)

# quote_df[quote_df['who'].str.contains('씨')]
quote_df[quote_df['who'] == '씨']

In [ ]:
quote_df[quote_df['doc_id'] == '109-0003944965']

In [ ]:
tmp_df = news_df[news_df['doc_id'] == '144-0000607710']

utils.get_quote(
    news_df=tmp_df,
    who_stop_list=who_stop_list,
    verb_stop_list=verb_stop_list,
)

In [ ]:
quote_df[quote_df['doc_id'] == '144-0000607710']

In [ ]:
quote_df[['doc_id', 'who', 'text', 'verb']]

In [ ]:
tmp = quote_df[['doc_id', 'verb']].groupby(by='verb').size().to_frame()
tmp[ tmp[0] > 4 ].sort_values(by=0, ascending=False)[:50]

In [ ]:
# lda

import matplotlib.pyplot as plt
import gensim
import numpy as np

from gensim.models import CoherenceModel, LdaModel, LsiModel, HdpModel
from gensim.models.wrappers import LdaMallet
from gensim.corpora import Dictionary
import pyLDAvis.gensim

import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.datasets import fetch_20newsgroups
from sklearn.decomposition import NMF, LatentDirichletAllocation

import os, re, operator, warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [ ]:
n_topics = 100
n_features = 1024

texts = []
for text in df['content'][:10000]:
    texts.append(text.replace('\n', ' '))

texts[1]

In [ ]:
stop_words = '10개 15일 1루수 1사 1승 1위 1패 2019 21일 22일 23일 24 24일 28일 29 2루수 2승 31 3루 3루수 3월 4월 5월 Copyright b컷 car chosun co com copyright copyrights donga goodnews http jtbc kbo kbo리그 khan kia kia의 kmib kr kt kt의 kyunghyang lg lg의 maekyung mk mk스포츠 mk스포츠배 mlb mtstarnews my mydaily nc news news1 now ops osen paper sk sk가 sk의 sports sportschosun sportsseoul sportsworldi spotv ufc vs www xportsnews x파일 yskim ⓒ 가장 개막 개막전 결과 공동 공식 광주 구단 구독하기 국민일보 그러나 그리고 금지 기록 기록했다 기사입니다 기사제보 기자 네이버 눈으로 뉴스 뉴스1 뉴스는 뉴시스 다만 단독 대전 독 동영상 두산 두산은 라고 롯데 리그 리얼타임 마이데일리 매경닷컴 메인에서 모든 모음전 무단 무단전재 무료만화 미리 및 바로가기 보기 보는 보도자료 부문 빨리 사주로 사진 사진=뉴시스 삼성은 생생 서울 속보 스타 스타뉴스 스타의 스포츠경향 스포츠서울 스포츠월드 스포츠조선 스포츠타임 스포탈코리아. 스포티비뉴스 시즌 신한은행 아이돌 안타를 않았다 알아보는 야구 야구대회 엑스포츠뉴스 엔트리에 엠스플뉴스 여기 역시 연속 연예스포츠 연합뉴스 열린 영상 오마이뉴스 올스타 올스타전 올시즌 올해 와이번스 운명의 웃음 유튜브 의뢰하세요 이번 인기 인기영상 일간스포츠 있는 있다 자료 작년 작성된 잠실 잠실구장에서 재배포 재배포금지 저작권자 전재 정규시즌 제2회 좋은 지금 지난 지난해 채널 쳤다 취재대행소 취재문의 취향저격 케이비리포트 클릭 타이거즈 팟캐스트 페이스북 평균 퓨처스 하다 하지만 한다 핫템 현장 현장에서'.split()

stop_words = list(set(stop_words))
print(' '.join(sorted(stop_words)))

tf_vectorizer = CountVectorizer(
    max_df=0.9,
    min_df=2,
    stop_words=stop_words,
    max_features=n_features,
)

tf = tf_vectorizer.fit_transform(texts)
tf_feature_names = tf_vectorizer.get_feature_names()

lda = LatentDirichletAllocation(
    n_topics,
    max_iter=10,
    learning_method='online',
    learning_offset=50.,
    random_state=0,
).fit(tf)

In [ ]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        words = [feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]

        print("Topic %d:\n" % (topic_idx))
        print(" ".join(words))
    return


display_topics(lda, tf_feature_names, 20)

In [ ]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)

In [ ]:
# df[ df['title'].str.contains('감독') ] 

search = df[ 
    (df['content'].str.contains('말했다.')) 
    | 
    (df['content'].str.contains('인터뷰')) 
]
search.info()

# 데이터 덤프

In [ ]:
news_list = utils.dump_data(
    host=host,
    auth=auth,
    doc_type='news',
    index_list=utils.index['news'], 
)

reply_list = utils.dump_data(
    host=host,
    auth=auth,
    doc_type='reply',
    index_list=utils.index['reply'], 
)

In [ ]:
utils.save_doc(index='naver-sports-news', data=news_list)
utils.save_doc(index='naver-sports-reply', data=reply_list)

# 데이터 변환

In [ ]:
# 데이터를 로딩한다.
news_list = utils.read_doc('naver-sports-news')
reply_list = utils.read_doc('naver-sports-reply')

In [ ]:
utils.get_simple_reply(doc_list=reply_list)
utils.merge_reply(news_list=news_list, reply_list=reply_list)

In [ ]:
reply_df = pd.DataFrame(reply_list)
reply_df.head()

In [ ]:
reply_df.info()

In [ ]:
news_df = pd.DataFrame(news_list)

news_df['date'].fillna(value='', inplace=True)
news_df['category'].fillna(value='', inplace=True)
news_df['reply_count'].fillna(value=0, inplace=True)

news_df.head()

In [ ]:
news_df.info()

In [ ]:
# 야구 분야만 추출한다.
kbo_df = news_df[news_df['category'].str.contains('야구')]

kbo_df['date'] = pd.to_datetime(kbo_df['date'])    

kbo_df.info()

# 통계 추출

In [ ]:
import numpy as np

utils.init_plt()

pd.set_option('precision', 2)
pd.set_option('display.float_format', '{:,.2f}'.format)

dt_range = None
# dt_range = {
#     'start': '2018-06-01',
#     'end': '2018-10-01',
# }

info = utils.get_reply_info(kbo_df=kbo_df, dt_range=dt_range)

In [ ]:
info['dt_range']

In [ ]:
info['reply_info'].head()

In [ ]:
info['reply_info'].loc['합계/평균'].to_frame().T

In [ ]:
info['reply_info'].drop(['합계/평균'])[['기사수', '댓글이 있는 기사']].plot(kind='barh', figsize=(15, 120))

In [ ]:
year = '2018'

date_df = info['reply_info'].drop(['합계/평균'])[['기사수', '댓글이 있는 기사']].reset_index()
for i in range(1, 7):
    st_date = '{}-{:02d}-01'.format(year, i)
    en_date = '{}-{:02d}-31'.format(year, i)
    
    mask = (date_df['date'] >= st_date) & (date_df['date'] <= en_date)
    date_df[mask].set_index('date').plot(kind='barh', figsize=(10, 10))

In [ ]:
info['reply_info'].drop(['합계/평균']).describe()

In [ ]:
# 데이터를 로딩한다.
news_list = utils.read_doc(utils.index['news'])
reply_list = utils.dump_reply(utils.index['reply'])

In [ ]:
reply_list = utils.dump_data(
    doc_type='reply',
    index_list=['crawler-naver-sports-reply'], 
)

In [ ]:
# 댓글에 날짜 삽입
reply_list = utils.insert_date(
    news_list=news_list, 
    reply_list=reply_list,
)

In [ ]:
doc = reply_list[1]
doc.keys()

In [ ]:
doc

In [ ]:
index = 'crawler-naver-sports-reply'

elastic_utils = ElasticSearchUtils(
    host='https://corpus.ncsoft.com:9200',
    index=index,
    bulk_size=100,
    http_auth='elastic:nlplab',
)

In [ ]:
for doc in tqdm(reply_list):
    if 'date' not in doc:
        continue

    elastic_utils.save_document(
        index=index,
        document={
            '_id': '{oid}-{aid}'.format(**doc),
            'date': doc['date'],
        }
    )
    
elastic_utils.flush()

In [ ]:
from elasticsearch import Elasticsearch

host = 'https://nlp.ncsoft.com:9200'
index = 'crawler-naver-sports-2018'
http_auth = 'crawler:crawler2019'

nlp_elastic = ElasticSearchUtils(
    host=host,
    index=index,
    bulk_size=100,
    http_auth=http_auth,
)

import pytz
from pprint import pprint
from dateutil.parser import parse as parse_date

timezone = pytz.timezone('Asia/Seoul')

for doc in tqdm(reply_list):
    if 'date' in doc:
        continue
    
    doc_id = '{oid}-{aid}'.format(**doc)
        
    flag = nlp_elastic.elastic.exists(
        id=doc_id,
        index=index,
    )

#     print(flag)
    if flag is True:
        news = nlp_elastic.elastic.get(
            id=doc_id,
            index=index,
        )['_source']
        
        if 'date' not in news and 'datetime' in news:
            dt = parse_date(news['datetime'])
            doc['date'] = timezone.localize(dt)
        
            elastic_utils.save_document(
                index='crawler-naver-sports-reply',
                document={
                    '_id': doc_id,
                    'date': doc['date'],
                }
            )
    
elastic_utils.flush()

In [ ]:
import pandas as pd
from utils.elasticsearch_utils import ElasticSearchUtils

host = 'https://nlp.ncsoft.com:9200'
index = 'crawler-naver-sports-2018'
http_auth = 'crawler:crawler2019'

elastic = ElasticSearchUtils(
    host=host,
    index=index,
    bulk_size=100,
    http_auth=http_auth,
)

query = {
    "_source": [
        "aid", "oid", "section", "category", "title", "date", "datetime", "url", "content"
    ],
    "query": {
        "bool": {
            "must": {
                "exists": {
                    "field": "category"
                }
            }
        }
    }
}

result = []
elastic.export(
    index=index,
    query=query,
    result=result,
)

df = pd.DataFrame(result)
df.head()